In [1]:
# Exercise 3.1 (a)
import numpy as np

def func3_1(x, c):
    return x**2 - c

def find_root3_1(c, init=1, tol=1.0e-8):
    if c == 0:
        return 0
    if c != 0 and init ==0:
        print('Initial value should be non-zero')
        return None

    root = init
    while abs(func3_1(root, c)) > tol:
        root = root / 2 + c / (2 * root)
    return root

**(b)** 
Since $f(0) < 0 \leq f(c)$ for $c \geq 1$ and $f(0) < 0 < f(1)$, a good starting value can be found by bisection method (with several iterations).

In [2]:
# Exercise 3.1 (b) and (c)
def find_start_value(c, niter=5):
    a = 0
    b = (c >= 1) * c + (c < 1) * 1
    for i in range(niter):
        d = (b - a) / 2
        if np.sign(func3_1(a + d, c)) == np.sign(func3_1(a, c)):
            a = a + d
        else:
            b = b - d
    return (a + b) / 2

def find_root3_1_sv(c, niter=5, tol=1.0e-8):
    sv = find_start_value(c, niter=niter)
    root = find_root3_1(c, init=sv, tol=tol)
    print('Starting value: {0:.8f}'.format(sv))
    print('Solution: {0:.8f}'.format(root))
    return root

In [3]:
# Exercise 3.2 (a)
def func3_2(x, c):
    try:
        return (x * (x + 2)) - c**2
    except OverflowError:
        return (x + c) * (x - c) + 2 * x

def find_root3_2(c, init=1, tol=1.0e-8):
    if c == 0:
        return 0
    root = init
    while abs(func3_2(root, c)) > tol:
        root = (3 * root - 1 / (1 + 1 / root)  + c * (c / (root + 1))) / 4
    return root

In [4]:
# Exercise 3.2 (b) and (c)
def find_start_value(c, niter=5):
    a = 0
    b = c
    for i in range(niter):
        d = (b - a) / 2
        if np.sign(func3_2(a + d, c)) == np.sign(func3_2(a, c)):
            a = a + d
        else:
            b = b - d
    return (a + b) / 2


def find_root3_2_sv(c, niter=5, tol=1.0e-8):
    sv = find_start_value(c, niter=niter)
    root = find_root3_2(c, init=sv, tol=tol)
    print('Starting value: {0:.8f}'.format(sv))
    print('Solution: {0:.8f}'.format(root))
    return root

In [5]:
# Exercise 3.3 (a)
from scipy.stats import norm

def BSVal(S, K, tau, r, delta, sigma):
    d = (np.log(np.exp(-delta * tau) * S) - np.log(np.exp(-r * tau) * K)) / (sigma * np.sqrt(tau)) + 0.5 * sigma * np.sqrt(tau)
    fval = np.exp(-delta * tau) * S * norm.cdf(d) - np.exp(-r * tau) * K * norm.cdf(d - sigma * np.sqrt(tau))
    dval = S * np.exp(-delta * tau) * np.sqrt(tau / (2 * np.pi)) * np.exp(-0.5 * d**2)
    return fval, dval

In [6]:
# Exercise 3.3 (b)
def ImpVol(S, K, tau, r, delta, V, init=1.0, tol=1.0e-8):
    def wrap_BSVal(sigma):
        f, df = BSVal(S, K, tau, r, delta, sigma)
        return f - V, df

    root, f, df = init, *wrap_BSVal(init)
    while abs(f) > tol:
        f, df = wrap_BSVal(root)
        root = root - f / df
    return root

print(ImpVol(100, 100, 1, 0.05, 0, 14.23))

0.29996692995674357


**Exercise 3.4**
The first order condition is
$$
A^{*}_{ij} = A_{ij} + \lambda_{i} d_{j}
$$
where $\lambda_{i}$ is the Lagrange multiplier on the $i$th constraint. Thus, the constrain gives
$$
\lambda_{i} = \frac{g_{i} - \sum_{j} A_{ij} d_{j}}{d^{T} d}
$$
and hence,
$$
A^{*}_{ij} = A_{ij} + \left(g_{i} - \sum_{j} A_{ij} d_{j}\right) \frac{d_{j}}{d^{T} d}
$$
In fact, the right hand side is the $(i, j)$ element of $A + (g - Ad) \frac{d^{T}}{d^{T} d}$.

**Exercise 3.5**
Consider the function $f: \mathbb{R}^2 \rightarrow \mathbb{R}^2$ defined by
$$
\begin{align}
    &f_{1}(x) = 200 x_{1} \left(x_{2} - x_{1}^{2}\right) - x_{1} + 1 \\
    & f_{2}(x) = 100 \left(x_{1}^{2} - x_{2} \right)
\end{align}
$$

Write a function that takes a column 2-vector $x$ as input and return a column 2-vector that contains the value of $f$ at $x$, and a 2-by-2 matrix that contain the Jacobian of $f$ at $x$.

- (a) Compute numerically the root fo $f$ using Newton's method
- (b) Compute numerically the root of $f$ using Broyden's method

In [7]:
# Exercise 3.5 (a)
def func3_5(x):
    f1 = x[0] * (x[1] - x[0]**2) - x[1] / 100 + 1 / 100
    f2 = x[0]**2 - x[1]
    return np.array([f1, f2])

def invJfunc3_5(x):
    df11 = 2 * x[1] - 6 * x[0]**2 - 1 / 100
    df12 = 2 * x[1]
    df21 = 2 * x[0]
    df22 = -1
    det = df11 * df22 - df12 * df21

    inv11, inv12, inv21, inv22 = df22, -df12, -df21, df11
    return np.array([[inv11, inv12], [inv21, inv22]]) / det

def newton3_5(init, tol=1.0e-10, maxiter=500):
    root = init
    for i in range(maxiter):
        fval, invJ = func3_5(root).T, invJfunc3_5(root)
        root = root - (invJ @ fval).T
        if np.dot(fval, fval) < tol:
            print('Convergence achieved with {:.0f} iterations'.format(i))
            print('Solution =', root)
            print('fval =', fval)
            break
        if i == maxiter - 1:
            print('Convergence not achieved')
            return root
    return root

sol3_5a = newton3_5([0.5, 0.5])

Convergence achieved with 43 iterations
Solution = [0.99995432 0.99990862]
fval = [-2.44674996e-06  5.44705659e-07]


In [8]:
# Exercise 3.5 (b)
def broyden3_5(init, tol=1.0e-10, maxiter=500):
    root_new, fval_new, invJ= init, func3_5(init).T, np.identity(len(init))
    for i in range(maxiter):
        root, fval = root_new, fval_new
        if np.dot(fval, fval) < tol:
            print('Convergence achieved with {:.0f} iterations'.format(i))
            print('Solution =', root)
            print('fval =', fval)
            break
        if i == maxiter - 1:
            print('Convergence not achieved')
            return root

        # Update root value and new function value
        root_new = root - (invJ @ fval).T
        fval_new = func3_5(root_new).T

        # Update inverse Jacobian
        d, u = root_new - root, invJ @ (fval_new - fval)
        invJ = invJ + np.outer(d - u, d.T) @ invJ / np.dot(d, u)
    return root_new

sol3_5b = broyden3_5([0.5, 0.5])

Convergence achieved with 311 iterations
Solution = [-0.99954548  0.9990918 ]
fval = [ 8.44533824e-06 -6.36930940e-07]


In [9]:
# Exercise 3.6
from scipy.stats import norm

def cdfnormal(x, mu, sigma):
    z = (x - mu) / sigma
    return [norm.cdf(z), norm.pdf(z)]

def icdf(p, F, x0, *args):
    if p <= 0 or p >= 1:
        print('p should be between 0 and 1')
        return None 
    x = x0
    Fval, dFval = F(*((x0,) + args))
    while abs(Fval - p) > 1.0e-8:
        Fval, dFval = F(*((x,) + args))
        x = x - (Fval - p) / dFval
    return x

icdf(0.975, cdfnormal, 0, 0, 1)

1.9599639845400536

In [10]:
# Exercise 3.7
from scipy import optimize


class Agent:

    def __init__(self, e, pref_a, pref_v):
        self.e = e
        self.pref_a = pref_a
        self.pref_v = pref_v

    def demand(self, p):
        M = np.dot(p, self.e)

        def auxfunc(x0):
            x1 = (M - p[0] * x0) / p[1]
            return self.pref_a[0] * x0**(self.pref_v[0]) - self.pref_a[1]  * (p[0] / p[1]) * x1**(self.pref_v[1])

        # Search initial value
        init0 = self.e[0]
        if init0 != 0:
            while (M - p[0] * init0) / p[1] <= 0:
                init0 = 0.99 * init0
        else:
            init1 = self.e[1]
            while (M - p[1] * init1) / p[0] <= 0:
                init1 = 0.99 * init1
            init0 = (M - p[1] * init1) / p[0]

        # Find optimal consumption bundle
        x0 = optimize.newton(auxfunc, init0, maxiter=100)
        x1 = (M - p[0] * x0) / p[1]
        return np.array([x0, x1])


class Market:

    def __init__(self, agents):
        self.agents = agents

    def aggrigate_net_demand(self, p0):
        # Check if the price vector is between 0 and 1
        if p0 > 1 or p0 < 0:
            raise ValueError('price should be between 0 and 1')

        p = np.array([p0, 1 - p0])
        agg_demand = np.sum([agent.demand(p) - agent.e for agent in self.agents], axis=0)
        return agg_demand[0]

    def equilibrium(self, init=0.5):
        p0 = optimize.newton(self.aggrigate_net_demand, x0=init, maxiter=500)
        return np.array([p0, 1 - p0])

# Preference parameters
_es = [[2.0, 3.0], [1.0, 2.0], [4.0, 0.0]]
_as = [[2.0, 1.5], [2.5, 2,0], [1.5, 2.0]]
_vs = [[-2.0, -0.5], [-1.5, -0.5], [-0.5, -1.5]]

# Generate market
agents = [Agent(np.array(e), np.array(a), np.array(v))
    for e, a, v in zip(_es, _as, _vs)]

market = Market(agents)
market.equilibrium()

array([0.26235095, 0.73764905])

In [11]:
# Exercise 3.8
def func3_8(c1, s, kappa=0.2, delta=0.95):
    return c1**(-5) + kappa - delta * (s - c1)**(-5)

def dfunc3_8(c1, s, kappa=0.2, delta=0.95):
    return -5 * c1**(-6) - 5 * delta * (s - c1)**(-6)

for s in [1.0, 2.0, 3.0]:
    c1 = optimize.newton(func3_8, x0=s/2, fprime=dfunc3_8, args=(s,))
    print('s = {0:.1f}: c1 = {1:.3f}'.format(s, c1))

s = 1.0: c1 = 0.503
s = 2.0: c1 = 1.026
s = 3.0: c1 = 1.712


**Exercise 3.9**
Provide a formal proof that the complementarity problem $CP(f, a, b)$ is equivalent to the rootfinding problem $\hat{f} = \min(\max(f(x), a - x), b - x) = 0$ in that both have the same solutions.

**Proof**
Suppose that $x^{*}$ is the solution of the complementarity problem $CP(f, a, b)$. If $x_{i}^{*} = a_{i}$, then $f_{i}(x_{i}^{*}) \geq 0 $ and $b_{i} - x_{i}^{*} \geq 0$. Hence, $\hat{f}_{i}(x_{i}^{*}) = 0$. If $x_{i}^{*} \in (a_{i}, b_{i})$, then $f_{i}(x) = 0$, $a_{i} - x_{i}^{*} < 0$ and $b_{i} - x_{i}^{*} > 0$. Hence, $\hat{f}_{i}(x^{*}) = 0$. If $x_{i}^{*} = b_i$, then $f_{i}(x^{*}) \leq 0$, $a_{i} - x_{i}^{*} \leq 0$. Hence, $\hat{f}(x^{*}) = 0$.

Suppose that $\hat{f}(x^{*}) = 0$. Then,
$$
0 = \hat{f}_{i} \left(x^{*}\right) =
\begin{cases}
    \max\left\{f\left(x^{*}\right), 0 \right\} & \text{if $x_{i}^{*} = a_{i}$} \\
    f\left(x^{*} \right) & \text{if $x_{i}^{*} \in (a, b)$} \\
    \min \left\{f \left(x^{*}\right), 0 \right\} & \text{if $x_{i}^{*} = b_{i}$}
\end{cases}
$$
Therefore, $f(x^{*}) = 0$.

**Exercise 3.10**
Commodity $X$ is produced and consumed in three countries. Let quantity $q$ be measured in units and price $p$ be measured in dollars per unit. Demand and supply in the countires are given by the following table:

||Demand|Supply|
|:---:|:---|:---|
|Country 1|$p=42-2q$|$p=9 + 1q$|
|Country 2|$p=54-3q$|$p=3 + 2q$|
|Country 3|$p=51-1q$|$p=18 + 1q$|

The unit costs of transportation are as follows:

|From\To|Country 1|Country 2|Country3|
|:---:|:---:|:---:|:---:|
|Country 1|0|3|9|
|Country 2|3|0|3|
|Country 3|6|3|0|

- (a) Formulate and solve the linear equation that characterizes competitive equilibrium, assuming that intercountry trade is not permitted
- (b) Formulate and solve the linear complementarity problem that characterizes competitive spatial equilibrium, assuming intercountry trade is permitted
- (c) Using standard measures of surplus, which of the six consumer and producer groups in the three countries gain, and which ones lose, from the introduction of trade?

**(a)**

- Country 1: $(p, q) = (20, 11)$
- Country 2: $(p, q) = (23.4, 10.2)$
- Country 3: $(p, q) = (34.5, 16.5)$

**(b)**
Let $q_{ij}$ be the amount of goods exported from $i$ to $j$. Then, the market clearing condition is
$$
\begin{align}
    &p_{j} = a_{j}^{D} + b_{j}^{D} \sum_{k=1}^{3} q_{kj} \\
    &p_{j} = a_{j}^{S} + b_{j}^{S} \sum_{k=1}^{3} q_{jk}
\end{align}
$$
and non-arbitrage requires
$$
\begin{align}
q_{ij} > 0 \Rightarrow p_{j} - p_{i} - c_{ij} = 0 \\
q_{ij} = 0 \Rightarrow p_{j} - p_{i} - c_{ij} \leq 0
\end{align}
$$
Hence, the complementarity problem is rewritten as the root finding problem
$$
0 = \max \left\{f(x), -x \right\}
$$
where $f_{ij}(x) = p_{j} - p_{i} - c_{ij}$.

In [12]:
# (b)
import numpy as np
from scipy import optimize

aD = np.array([42, 54, 51])
aS = np.array([9, 3, 18])
bD = np.array([-2, -3, -1])
bS = np.array([1, 2, 1])
c = np.array([[0, 3, 9], [3, 0, 3], [6, 3, 0]])

def priceD(x, i):
    return aD[i] + bD[i] * np.sum(x.T[i])

def priceS(x, i):
    return aS[i] + bS[i] * np.sum(x[i])

def func3_10(x):
    return np.array([[max(priceD(x, j) - priceS(x, i) - c[i][j], -x[i][j]) for j in range(3)] for i in range(3)])

def wrap_func3_10(x):
    fval = func3_10(x.reshape(3, 3))
    return np.array([fval[i][j] for i in range(3) for j in range(3)])

x0 = np.array([11, 0, 0, 0, 10.2, 0, 0, 0, 16.5])
q = optimize.newton(wrap_func3_10, x0, maxiter=1000).reshape(3, 3)
p = np.array([priceD(q, i) for i in range(3)])
print(p)
print(q)

[23.99998166 26.99998626 30.00000917]
[[ 9.00000917  5.99997707  0.        ]
 [ 0.          3.00002751  8.99997707]
 [ 0.          0.         12.00001375]]


**(c)**
Omitted.
